In [8]:
import os
import re
import random
import json
# Mount to google drive
from google.colab import drive
drive.mount('/content/drive')

# Change it to your google drive path where this notebook located.
drive_path = '/content/drive/MyDrive/Projects/CryptoniteAnalysis/'
os.chdir(drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
def split_explanations_to_files(input_file):
    # Open the input file
    with open(input_file, 'r', encoding='utf-8-sig') as f:
        content = f.read()

    # Split the content by two or more blank lines (paragraph separator)
    explanations = content.split('\n\n')

    # Remove any leading/trailing whitespace in each paragraph
    explanations = [para.strip() for para in explanations if para.strip()]

    return explanations

def extract_clues_and_orientation(explanations):
    pattern = r"\*\*Clue\*\*:\s*(.*)"
    clues = []
    # Loop through each paragraph, extract the clue, and save to separate files
    for i, paragraph in enumerate(explanations, 1):
        match = re.match(pattern, paragraph)
        if match:
            # Extract the part after "Clue"
            clue_part = match.group(1)
            clues.append(clue_part)

    return clues

def create_json_for_clues(explanations, clues):
    first_hard_clue = 'It’s crazy changing what comes before pi (7)'
    down_clues = ['Put out by mounting objections (4)',
                  'Festive occasion was put on canvas (7)']
    quick = True
    all_clue_json = []
    for i, clue in enumerate(clues):
        if clue == first_hard_clue:
            quick = False
        orientation = random.choice(['across', 'down'])
        if clue in down_clues:
            orientation = 'down'

        clue_json = {'clue': clue,
                     'orientation': orientation,
                     'quick': quick,
                     # input for LLM (for fine tuning)
                     'input': f"**Clue**: {clue}\n**Orientation**: {orientation}",
                     'explanation': explanations[i]}
        all_clue_json.append(clue_json)
    return all_clue_json


In [12]:
# Example usage
input_file = 'PromptEngineering/InContextLearningExamples/HandcraftedExamples.txt'
explanations = split_explanations_to_files(input_file)
clues = extract_clues_and_orientation(explanations)
all_json = create_json_for_clues(explanations, clues)

In [14]:
with open('PromptEngineering/InContextLearningExamples/ProcessedExamples.jsonl', 'w') as f:
    for entry in all_json:
        # Convert each dictionary to a JSON string and write it to the file
        f.write(json.dumps(entry) + '\n')